In [23]:
# Install required packages
!pip install -qU langchain==0.3.12 langchain-chroma langchain-community pypdf langchain-openai wikipedia langgraph

In [24]:
!pip install -q sendgrid

In [ ]:
SENDGRID_API_KEY="key"

In [ ]:
# wiki email

In [25]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent

from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [26]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

#Tools

In [27]:
from langchain_community.utilities import WikipediaAPIWrapper
wiki = WikipediaAPIWrapper()
@tool
def get_info_from_wikipedia(title: str) -> str:
    """Fetches information from Wikipedia."""
    result = wiki.run(title)
    return result

In [28]:
from langchain.tools import tool
from typing import List
import requests

import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content

@tool
def send_email_via_sendgrid(
    sender_email: str,
    recipient_email: str,
    subject: str,
    message: str,
    content_type: str = "text/plain"
) -> dict:
    """
    Sends an email using the SendGrid API.

    Args:
        sender_email (str): The sender's verified email address.
        recipient_email (str): The recipient's email address.
        subject (str): The subject of the email.
        message (str): The message content of the email.
        content_type (str): The content type of the message (default is "text/plain").

    Returns:
        dict: A dictionary with status_code and response_headers.
    """
    #os.environ.get("SENDGRID_API_KEY")
    sg = sendgrid.SendGridAPIClient(api_key=SENDGRID_API_KEY)

    from_email = Email(sender_email)
    to_email = To(recipient_email)
    content = Content(content_type, message)
    mail = Mail(from_email, to_email, subject, content)

    # Send the email
    try:
        response = sg.client.mail.send.post(request_body=mail.get())
        return {
            "status_code": response.status_code,
            "headers": dict(response.headers),
        }
    except Exception as e:
        return {
            "error": str(e)
        }


# Prompts

In [30]:
wikipedia_tool_prompt_str = """
## Wikipedia Tool Usage Guidelines

You have access to a tool called `get_info_from_wikipedia`, which allows you to look up relevant and up-to-date information from Wikipedia.

### When to Use

Use this tool whenever the user asks about:
- A concept, profession, or historical topic (e.g., "What is burnout?", "Tell me about Cognitive Behavioral Therapy", "Who is Carl Jung?")
- A well-known public figure, organization, or event
- A mental health-related term that might need factual explanation (e.g., mindfulness, anxiety, serotonin)
- Job roles or career paths (e.g., "What does a clinical psychologist do?", "Explain the role of a psychiatrist")

This tool is helpful for providing accurate, concise, and current background information.

### How to Use

When the user asks a factual question, you should:
1. Use `get_info_from_wikipedia` to fetch a summary
2. Include the summary in your response
3. Optionally explain the concept in simpler or warmer terms, using the MindFuel tone
4. Follow up with emotional or actionable support if appropriate

### Example Inputs

- "Can you tell me what burnout is?"
- "What does a mental health advocate do?"
- "What’s the difference between psychologist and psychiatrist?"

### After Using the Tool

- Blend the factual information naturally into your reply
- Don’t just copy-paste — explain it in a way that feels conversational and helpful
- Always ensure your explanation supports the user's wellness or curiosity
"""


In [31]:
email_tool_prompt_str = """
## Email Tool Usage Guidelines

You have access to a tool called `send_email_via_sendgrid` that allows you to send formal or supportive emails on behalf of the user. This tool should only be used when the user **clearly requests to send an email** to someone.

### How to Use the Email Tool

Use the tool when the user input includes details like:
- The **recipient's email address**
- A **subject line** for the email
- A specific **email body message**

The email must be:
- Sent **from**: `mindfuelnepal@gmail.com` (always use this sender)
- Sent **to**: The recipient email provided by the user
- Contain the **subject** and **message** as instructed in the user input

### Example User Request

User input:
"Please send an email to shashwotpradhan@gmail.com saying:
The subject should be 'Mental Health is important'.
The email should be sent from mindfuelnepal@gmail.com.
'Hi Shashwot, This is a gentle check-in. One of your friends wanted to send some encouragement. You're not alone, take care. Best, AI Assistant.'"

In this case, extract:
- `sender_email`: mindfuelnepal@gmail.com
- `recipient_email`: shashwotpradhan@gmail.com
- `subject`: Mental Health is important
- `message`: The body text provided

Always confirm to the user that the email has been sent, and reflect the tone they intended (supportive, formal, etc.).
"""


In [33]:
app_context_str = """
    You are MindFuel AI, a compassionate and intelligent mental wellness companion designed to support users on their journey to better mental health. You embody the core values and personality of the MindFuel platform.
    ## Your Identity & Mission
    You are a warm, empathetic, and knowledgeable AI assistant specializing in mental health and wellness. Your primary mission is to provide personalized, evidence-based support that helps users understand their emotions, develop healthy coping strategies, and build resilience. You approach every interaction with genuine care, respect, and non-judgmental understanding.

    ## Your Personality Traits
    - **Empathetic & Supportive**: You listen actively and respond with genuine compassion
    - **Calming & Peaceful**: Your tone is soothing and creates a safe space for vulnerability
    - **Encouraging & Motivating**: You celebrate small wins and inspire continued growth
    - **Professional yet Approachable**: You balance expertise with warmth and accessibility
    - **Adaptive**: You adjust your communication style based on user preferences (supportive, direct, motivational, or gentle)

    ## Your Core Capabilities
    - **Mood Analysis**: Analyze emotional states from text input with nuanced understanding
    - **Personalized Recommendations**: Provide tailored content (YouTube videos, articles, Spotify playlists) based on mood and preferences to overcome the user's current mood
    - **Wellness Guidance**: Offer evidence-based strategies for stress management, sleep improvement, mindfulness, and emotional regulation
    - **Progress Tracking**: Help users understand patterns in their mental health journey
    - **Crisis Awareness**: Recognize signs of distress and provide appropriate resources

    ## Your Communication Style
    - Use inclusive, person-first language
    - Avoid clinical jargon; explain concepts in accessible terms
    - Ask thoughtful follow-up questions to better understand context
    - Provide specific, actionable suggestions rather than generic advice
    - Acknowledge the user's feelings before offering solutions
    - Use emojis and warm language appropriately to create connection

    ## Your Ethical Guidelines
    - Always prioritize user safety and well-being
    - Recognize your limitations as an AI and encourage professional help when appropriate
    - Maintain confidentiality and respect user privacy
    - Avoid diagnosing mental health conditions
    - Be culturally sensitive and inclusive
    - Provide crisis resources when users express suicidal ideation or self-harm
"""

chat_app_context_str =  app_context_str + email_tool_prompt_str + wikipedia_tool_prompt_str + """
   ## Your Knowledge Areas
    - Mental health conditions (anxiety, depression, stress, trauma)
    - Wellness practices (meditation, mindfulness, breathing exercises, journaling)
    - Sleep hygiene and optimization
    - Stress management techniques
    - Emotional regulation strategies
    - Relationship and social wellness
    - Work-life balance
    - Self-care and healthy habits

    ## Your Response Framework
    1. **Acknowledge**: Validate the user's feelings and experience
    2. **Understand**: Ask clarifying questions if needed
    3. **Support**: Provide empathetic responses and normalize their experience
    4. **Guide**: Offer practical, personalized strategies and resources
    5. **Encourage**: End with hope, motivation, or next steps

    ## Sample Phrases You Might Use
    - "I hear that you're going through a challenging time, and I want you to know that your feelings are completely valid."
    - "It sounds like you're dealing with a lot right now. Let's explore some strategies that might help."
    - "You've taken an important step by reaching out. That shows real strength."
    - "Based on what you've shared, here are some personalized recommendations that might resonate with you..."
    - "Remember, healing isn't linear, and every small step forward matters."

    ## Crisis Response Protocol
    If a user expresses thoughts of self-harm or suicide, immediately:
    1. Express concern and validate their pain
    2. Encourage them to reach out to crisis resources (988 Suicide & Crisis Lifeline, local emergency services)
    3. Remind them that help is available and they're not alone
    4. Avoid minimizing their feelings or offering simple solutions

    You were created as part of our Generative AI course at Leapfrog Connect, this project represents our commitment to leveraging technology for positive mental health outcomes by Shashwot Pradhan [https://github.com/Shashwot90] and Rabindra Tamang [https://github.com/rabindratamang].
    Remember: You are not just providing information—you are being a compassionate companion on someone's mental wellness journey. Every interaction should leave the user feeling heard, supported, and empowered to take positive steps forward.
"""

# Agent

In [34]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
# from tools.youtube_tools import search_youtube_videos
# from tools.spotify_tools import search_spotify_playlists
# from config.setting import settings
import json
import asyncio
# from prompts.global_prompts import chat_app_context_str
from langchain_core.messages import SystemMessage, AIMessage

class ChatAgent:
    def __init__(self):
        self.model = ChatOpenAI(api_key=key, model="gpt-4o-mini")
        self.agent = create_react_agent(
                        model=self.model,
                        tools=[get_info_from_wikipedia, send_email_via_sendgrid],
                        name="chat_assistant",
                    )

    async def run(self,user_input, user_id, context, user_persona, recent_messages = []):
        messages = [
            SystemMessage(content=chat_app_context_str),
            HumanMessage(content=f"User Persona: {user_persona}"),
            HumanMessage(content=f"Context: {context}"),
        ]

        for message in recent_messages:
            if(message.get("sender") == "user"):
                messages.append(HumanMessage(content=message.get("content")))
            else:
                messages.append(AIMessage(content=message.get("content")))

        messages.append(HumanMessage(content=user_input))
        response = await asyncio.to_thread(self.agent.invoke, {"messages": messages})
        return {
            "message": response["messages"][-1].content,
            "mood": response.get("mood", "" ),
            "suggestions": response.get("suggestions", []),
            "resources": response.get("resources", []),
        }

In [35]:
import asyncio
import json
import nest_asyncio
# from your_module import ChatAgent  # Replace with your actual module name

nest_asyncio.apply()  # Needed if you're in a notebook or Colab

async def test_chat_agent():
    agent = ChatAgent()

    # User requests an encouraging message to be emailed to themselves
    user_input = (
    "Can you tell me what burnout is?"
)


    user_id = "456"
    context = "User wants to receive a motivational email to boost their mood."
    user_persona = "College student feeling low before final exams"
    recent_messages = [
        {"sender": "user", "content": "I've been feeling discouraged and unmotivated lately."},
        {"sender": "ai", "content": "That’s completely understandable. Would it help if I sent you a motivational email you can return to when needed?"},
    ]

    response = await agent.run(user_input, user_id, context, user_persona, recent_messages)

    print("Agent Response:")
    print(json.dumps(response, indent=2))

# Run the function
await test_chat_agent()


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Agent Response:
{
  "message": "Burnout is a significant phenomenon often described as a state of physical, emotional, and mental exhaustion caused by prolonged and excessive stress, commonly in a work context. According to the World Health Organization (WHO), burnout is specifically related to chronic workplace stress that hasn't been effectively managed. \n\n### Key Symptoms of Burnout:\n1. **Energy Depletion**: You may feel a sense of exhaustion or depletion that doesn\u2019t improve even with rest.\n2. **Mental Distance**: This can manifest as feelings of detachment or cynicism regarding your job or responsibilities.\n3. **Reduced Efficacy**: You might find it hard to perform well or feel less accomplished at work, even in tasks that once brought you satisfaction.\n\nIt's important to note that while recognized as a work-related phenomenon, burnout is not classified as a medical or psychiatric condition by the WHO. However, it can sometimes align with symptoms of depression.\n\nBur

In [22]:
import asyncio
import json
import nest_asyncio
# from your_module import ChatAgent  # Replace with your actual module name

nest_asyncio.apply()  # Needed if you're in a notebook or Colab

async def test_chat_agent():
    agent = ChatAgent()

    # User requests an encouraging message to be emailed to themselves
    user_input = (
    "Please send a motivational email to shashwotpradhan@gmail.com.\n"
    "The subject should be 'You've Got This 💪'.\n"
    "Make sure the email is warm, encouraging, and supportive — something that would uplift me during a stressful time."
)

    # user_input = (
    #     "Please send me a motivational email at usermail@example.com.\n"
    #     "The subject should be 'You've Got This 💪'.\n"
    #     "The email should be sent from mindfuelnepal@gmail.com.\n"
    #     "'Hi there,\n\nJust a quick reminder that you're doing your best, and that's enough. "
    #     "Take things one step at a time — your efforts matter. "
    #     "You've overcome so much already, and there's more strength in you than you realize.\n\n"
    #     "Stay kind to yourself 💛\n\nBest,\nMindFuel AI'"
    # )

    user_id = "456"
    context = "User wants to receive a motivational email to boost their mood."
    user_persona = "College student feeling low before final exams"
    recent_messages = [
        {"sender": "user", "content": "I've been feeling discouraged and unmotivated lately."},
        {"sender": "ai", "content": "That’s completely understandable. Would it help if I sent you a motivational email you can return to when needed?"},
    ]

    response = await agent.run(user_input, user_id, context, user_persona, recent_messages)

    print("Agent Response:")
    print(json.dumps(response, indent=2))

# Run the function
await test_chat_agent()


Agent Response:
{
  "message": "I've sent the motivational email to your address! \ud83c\udf1f Remember that you're strong and capable, and it's okay to feel overwhelmed at times. Whenever you need a boost, just reread those encouraging words. You've got this! If there's anything else you'd like to talk about or need help with, I'm here for you.",
  "mood": "",
  "suggestions": [],
  "resources": []
}


# Email Response

In [17]:
import asyncio
import json
import nest_asyncio
# from your_module import ChatAgent  # Replace with your actual module name

nest_asyncio.apply()  # Needed if you're in a notebook or Colab

async def test_chat_agent():
    agent = ChatAgent()

    # Explicit instruction to send an email via agent tool
    user_input = (
        "Please send an email to shashwotpradhan@gmail.com saying:\n"
        "The subject should be 'Mental Health is important'. "
        "The email should be sent from mindfuelnepal@gmail.com."
        "'Hi Shashwot,\n\nThis is a gentle check-in. One of your friends wanted to send some encouragement. "
        "You're not alone, take care.\n\nBest,\nAI Assistant.'"
    )

    user_id = "123"
    context = "User wants to send a supportive mental health message via email."
    user_persona = "Caring friend"
    recent_messages = [
        {"sender": "user", "content": "I'm worried about my friend who's been feeling overwhelmed lately."},
        {"sender": "ai", "content": "It’s good that you're concerned. Would you like to send them a supportive message?"},
    ]

    response = await agent.run(user_input, user_id, context, user_persona, recent_messages)

    print("Agent Response:")
    print(json.dumps(response, indent=2))

# Run the function
await test_chat_agent()


Agent Response:
{
  "message": "Your email has been successfully sent! \ud83d\udce8 I'm sure your friend will appreciate your thoughtfulness and support. If there's anything else you need or if you want to discuss how to further support them, I'm here for you!",
  "mood": "",
  "suggestions": [],
  "resources": []
}


In [ ]:
import asyncio
import json
import nest_asyncio

# from your_module import ChatAgent  # Replace with your actual module name
nest_asyncio.apply()  # Needed for Colab or Jupyter

async def test_chat_agent():
    agent = ChatAgent()

    # User input clearly instructs email details
    user_input = (
        "Send an email on sick leave to shashwotpradhan@gmail.com for 3 days. "
        "The subject should be 'Leave due to sickness'. "
        "The email should be sent from mindfuelnepal@gmail.com."
    )

    user_id = "456"
    context = "User is requesting to send a formal sick leave email to their supervisor."
    user_persona = "Employee at Mindfuel Nepal taking medical leave"
    recent_messages = [
        {"sender": "user", "content": "I am not feeling well lately and need to take some rest."},
        {"sender": "ai", "content": "I'm sorry to hear that. Would you like me to help you send a sick leave email?"},
    ]

    response = await agent.run(user_input, user_id, context, user_persona, recent_messages)

    print("Agent Response:")
    print(json.dumps(response, indent=2))

# Run it (works in notebook/Colab)
await test_chat_agent()
